OneTwoTrip предлагает решить две задачи по предсказанию поведения клиентов на сайте. Датасет общий на обе задачи и содержит анонимизированную информацию о пользователях и их действиях на сайте.
Медали и рейтинг в общий зачет присуждаются отдельно за каждую задачу. Финальный лидерборд строится после публикации приватного рейтинга по обеим задачам. Участники, занявшие 1-10 места в задачах, получат от 10 до 1 балла соответственно за каждую задачу, после баллы суммируются. При равенстве баллов у участников приоритет отдается решению второй задачи.

Задача 1:
Если у пассажира изменились планы или по каким-то причинам он не сможет воспользоваться перелетом, то он может вернуть авиабилет. Для этого ему надо создать заявку на возврат. Для каждого заказа (orderid) предскажите с какой вероятностью пользователь подаст заявку на возврат билета. В качестве сабмита принимается .csv файл с двумя колонками - id заказа (orderid) и вероятностью возврата. 

In [ ]:
#загрузка необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score,GroupKFold
from sklearn import preprocessing
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
%matplotlib inline
from IPython.core.display import display, HTML
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
#загружаем train и test
train = pd.read_csv('onetwotrip_challenge_train.csv')
train = train.sort_values(['userid','field4']).reset_index(drop=True) # сортируем трейн по юзерам и номерам их покупок билетов
test = pd.read_csv('onetwotrip_challenge_test.csv')

#кодируем айдишники юзеров числами
le = preprocessing.LabelEncoder()
le.fit(pd.concat([train['userid'],test['userid']]))
train['userid'] = le.transform(train['userid'])
test['userid'] = le.transform(test['userid'])

In [ ]:
fff = [x for x in train.columns if x not in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]

In [ ]:
# В датасете данные за 2 года, отдельная фича означает 0 - первый год, 1 - второй. 
# Добавим фичи с месяцами покупки и вылета ко второму году

for df in [train,test]:
    df.loc[df['field21']==1,'field2'] = df['field2'] + 12 # Добавим фичу с месяцем покупки ко второму году
    df.loc[df['field21']==1,'field3'] = df['field3'] + 12 # Добавим фичу с месяцем вылета ко второму году
    df['ticket_price'] = df['field1'] / df['field15'] # средняя цена одного билета в покупке
    
    df['userid'] = le.transform(df['userid']) # не конкатенируем, т.к. юзеры трейна и теста не пересекаются
    df['COUNT_id'] = df['userid'].map(df['userid'].value_counts()) # заменим userid на каунты


#Посчитаем некоторые эффективные статистики: 
col = 'userid'
for df in [train,test]:
    for f in ['ticket_price']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std()) #del train['ticket_price'],test['ticket_price']
   
    for f in ['field16','field0','field23']: #22,11,13,1,12,price,3,6,20,2,18
        df['{}_max'.format(f)] = df[col].map(df.groupby(col)[f].max())
        df['{}_min'.format(f)] = df[col].map(df.groupby(col)[f].min())
        df['{}_razmah'.format(f)] = df['{}_max'.format(f)]- df['{}_min'.format(f)]
        del df['{}_min'.format(f)],df['{}_max'.format(f)]
        
for df in [train,test]:
    for f in ['field26']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
        df['{}_{}_MEAN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].mean())
        df['{}_{}_MAX'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].max())

for df in [train,test]:
    for f in ['field0']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
for df in [train,test]:
    for f in ['field13']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
for df in [train,test]:
    for f in ['field1']:
        df['{}_{}_MEAN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].mean())
for df in [train,test]:
    for f in ['field16']:
        df['{}_{}_MEAN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].mean())
        df['{}_{}_MIN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].min())
for df in [train,test]:
    for f in ['field12']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
for df in [train,test]:        
    for f in ['field23','ticket_price']:
        df['first_ticket_{}'.format(f)] = df['userid'].map(df[df['field4']==1][['userid',f]].set_index('userid')[f].to_dict())

In [ ]:
#Брутфорс поиска эффективных фич выявил что каунты категориальной фичи: 'field13' + 'field22' дают ощутимый прирост качеству модели.

for f1 in ['field13']:
    for f2 in ['field22']:
        if f1 != f2:
            train['COUNT_{}_{}'.format(f1,f2)] = train[f1].astype(str) + "_" + train[f2].astype(str)
            test['COUNT_{}_{}'.format(f1,f2)] = test[f1].astype(str) + "_" + test[f2].astype(str)
            train['COUNT_{}_{}'.format(f1,f2)] = train['COUNT_{}_{}'.format(f1,f2)].map(pd.concat([train['COUNT_{}_{}'.format(f1,f2)],test['COUNT_{}_{}'.format(f1,f2)]]).value_counts())
            test['COUNT_{}_{}'.format(f1,f2)] = test['COUNT_{}_{}'.format(f1,f2)].map(pd.concat([train['COUNT_{}_{}'.format(f1,f2)],test['COUNT_{}_{}'.format(f1,f2)]]).value_counts())
            
for f in ['field12','field16']:  
    train['COUNT_{}'.format(f)] = train[f].map(pd.concat([train[f],test[f]]).value_counts())
    test['COUNT_{}'.format(f)] = test[f].map(pd.concat([train[f],test[f]]).value_counts())        

In [ ]:
# уменьшим размер нашего датасета
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#Добавим отдельными признаками некоторые данные о первых покупках билетов и о последнем билете

for f in ['field0','field16']:
    for n in range(2,4):
        train['{}_ticket_{}'.format(n,f)] = train['userid'].map(train[train['field4']==n][['userid',f]].set_index('userid')[f].to_dict())
        test['{}_ticket_{}'.format(n,f)] = test['userid'].map(test[test['field4']==n][['userid',f]].set_index('userid')[f].to_dict())
        train = reduce_mem_usage(train)
        test = reduce_mem_usage(test)
    print(f)

In [ ]:
# переведём некоторые дискретные признаки в набор бинарных признаков
train = pd.get_dummies(data=train,columns=['field27'])
test = pd.get_dummies(data=test,columns=['field27'])

#укажем NaN для тех статистик покупок, которых не более 1 у юзера
y=2
for col in train.columns:
    if ('MIN' in col) or ('MAX' in col) or ('STD' in col) or ('MEAN' in col):
        train.loc[train['COUNT_id']<y,col] = np.nan
        test.loc[test['COUNT_id']<y,col] = np.nan

In [ ]:
# target-encoding хорошо сработал для фичи field18

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone
from sklearn.model_selection import check_cv
from category_encoders import CatBoostEncoder

y_train = train['goal1'].copy()
X_train = train.drop(columns = ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid'], axis=1)
X_test = test.copy()

class TargetEncoderCV(BaseEstimator, TransformerMixin):
    def __init__(self, cv, **cbe_params):
        self.cv = cv
        self.cbe_params = cbe_params
    @property
    def _n_splits(self):
        return check_cv(self.cv).n_splits
    def fit_transform(self, X: pd.DataFrame, y) -> pd.DataFrame:
        self.cbe_ = []
        cv = check_cv(self.cv)
        cbe = CatBoostEncoder(
            cols=X.columns.tolist(),
            return_df=False,
            **self.cbe_params)
        X_transformed = np.zeros_like(X, dtype=np.float64)
        for train_idx, valid_idx in cv.split(X, y):
            self.cbe_.append(
                clone(cbe).fit(X.loc[train_idx], y[train_idx]))
            X_transformed[valid_idx] = self.cbe_[-1].transform(X.loc[valid_idx])
        return pd.DataFrame(X_transformed, columns=X.columns)
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X_transformed = np.zeros_like(X, dtype=np.float64)
        for cbe in self.cbe_:
            X_transformed += cbe.transform(X) / self._n_splits
        return pd.DataFrame(X_transformed, columns=X.columns)

for trans_feature in ['field18']:
    y_train = train['goal1'].copy()
    X_train = train.drop(columns = ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid'], axis=1)
    X_test = test.copy()
    X_train[trans_feature] = X_train[trans_feature].astype(str)
    X_test[trans_feature] = X_test[trans_feature].astype(str)
    te_cv = TargetEncoderCV(KFold(n_splits=3))
    X_train = te_cv.fit_transform(X_train[[trans_feature]], y_train)
    X_test = te_cv.transform(X_test[[trans_feature]])
    train[trans_feature] = X_train[trans_feature]
    test[trans_feature] = X_test[trans_feature]
del X_train,X_test
gc.collect()

In [ ]:
features = [c for c in train.columns if c not in ['field1','goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]

N=5
rlist = [59,73,42,65,78]

for r in rlist:
    x_pred = np.zeros(train[features].shape[0])
    y_pred = np.zeros(test[features].shape[0])
    
    #равномерно распределим записи в датасете, чтобы дисбаланс в каждом фолде был одинаковый
    train = train.sort_values('goal1').reset_index(drop=True)
    vc = train['goal1'].value_counts()
    vc = dict(sorted(vc.items()))
    df = pd.DataFrame()
    train['indexcol'],i = 0,1
    for k,v in vc.items():
        step = train.shape[0]/v
        indent = train.shape[0]/(v+1)
        df2 = train[train['goal1'] == k].sample(v,random_state = r).reset_index(drop=True)
        for j in range(0, v):
            df2.at[j, 'indexcol'] = indent + j*step + 0.000001*i
        df = pd.concat([df2,df])
        i+=1
    train = df.sort_values('indexcol', ascending=False).reset_index(drop=True)
    del train['indexcol'], df, df2, vc
    
    #фолды делим kfold'ом
    folds = GroupKFold(n_splits=N)
    target = train['goal1'].copy()

    lgb_param = {
        'max_bin':24,
        'min_data_in_leaf': 8,
        'num_leaves': 128,
        'learning_rate': 0.01,
        'min_child_weight': 0.3,
        'bagging_fraction': 0.4, 
        'feature_fraction': 0.595,
        'max_depth': 7, 
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric':'auc'}

    importance = pd.DataFrame(np.zeros((train[features].shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train[features].columns)
    scores = []
    oof = np.zeros(train[features].shape[0])

    for fold, (trn_idx, val_idx) in enumerate(folds.split(train[features], target), 1):
        print('Fold {}'.format(fold))
        trn_data = lgb.Dataset(train[features].iloc[trn_idx, :].values, label=target.iloc[trn_idx].values)
        val_data = lgb.Dataset(train[features].iloc[val_idx, :].values, label=target.iloc[val_idx].values)   
        clf = lgb.train(lgb_param, trn_data, 500, valid_sets=[trn_data, val_data], verbose_eval=500, early_stopping_rounds=101)
        predictions = clf.predict(train[features].iloc[val_idx, :].values) 
        importance.iloc[:, fold - 1] = clf.feature_importance()
        oof[val_idx] = predictions
        score = roc_auc_score(target.iloc[val_idx].values, predictions)
        scores.append(score)
        y_pred += clf.predict(test[features]) / (N*len(rlist))
        x_pred += clf.predict(train[features]) / (N*len(rlist))
        del trn_data, val_data, predictions
        gc.collect()

    print(r, '######### Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))
    gc.collect()


In [ ]:
importance['Mean_Importance'] = importance.sum(axis=1) / N
importance = importance.sort_values(by='Mean_Importance', ascending=False)[:50]
plt.figure(figsize=(15, 15))
sns.barplot(x='Mean_Importance', y=importance.index, data=importance)
plt.xlabel('')
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.title('Mean Feature Importance Between Folds', size=15)
plt.savefig('{}.png'.format(sum(scores)))
plt.show()

In [ ]:
# формируем файл с предсказаниями
pd.DataFrame(y_pred, columns=['proba'], index=test['orderid']).to_csv('submission.csv')

### Catboost

In [ ]:
features = [c for c in train.columns if c not in ['field1','goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]

N=5
rlist = [59,73,42]

rlist = range(5)

y_test_pred = 0
for r in rlist:


    train = pd.read_pickle('tr_1')
    test = pd.read_pickle('te_1')
    features = [c for c in train.columns if c not in ['field1','goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]
    
    train = train.sort_values('goal1').reset_index(drop=True)
    vc = train['goal1'].value_counts()
    vc = dict(sorted(vc.items()))
    df = pd.DataFrame()
    train['indexcol'],i = 0,1
    for k,v in vc.items():
        step = train.shape[0]/v
        indent = train.shape[0]/(v+1)
        df2 = train[train['goal1'] == k].sample(v,random_state = r).reset_index(drop=True)
        for j in range(0, v):
            df2.at[j, 'indexcol'] = indent + j*step + 0.000001*i
        df = pd.concat([df2,df])
        i+=1
    train = df.sort_values('indexcol', ascending=False).reset_index(drop=True)
    del train['indexcol'], df, df2, vc

    folds = KFold(n_splits=N)
    target = train['goal1'].copy()

    y_valid_pred = 0 * target
  
    for idx, (train_index, valid_index) in enumerate(kf.split(train)):

        model = CatBoostClassifier(loss_function="Logloss",
               eval_metric="AUC",
               task_type="GPU",
               learning_rate=0.01,
               iterations=10000,
               od_type="Iter",
               depth=7,
               early_stopping_rounds=300)
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        X_train, X_valid = train[features].iloc[train_index,:], train[features].iloc[valid_index,:]
        _train = Pool(X_train, label=y_train)
        _valid = Pool(X_valid, label=y_valid)
        print( "\nFold ", idx)
        fit_model = model.fit(_train,
                              eval_set=_valid,
                              use_best_model=True,
                              verbose=1000,
                              plot=True
                             )
        pred = fit_model.predict_proba(X_valid)[:,1]
        print( "  auc = ", roc_auc_score(y_valid, pred) )
        y_valid_pred.iloc[valid_index] = pred
        y_test_pred += fit_model.predict_proba(test[features])[:,1]
        gc.collect()
        
y_test_pred /= (N*len(rlist))
        
gc.collect()

In [ ]:
pd.DataFrame(y_test_pred, columns=['proba'], index=test['orderid']).to_csv('catboost.csv')